In [ ]:
import numpy as np 
import pandas as pd

import io
import os
from os import listdir
from os.path import isfile, join
import pickle
import itertools
from pymatreader import read_mat

from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score , roc_auc_score, recall_score,precision_score
from sklearn import metrics
from sklearn.metrics import multilabel_confusion_matrix

#import classifiers from scikit-learn
from sklearn.ensemble import AdaBoostClassifier,BaggingClassifier,ExtraTreesClassifier,GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier,StackingClassifier,VotingClassifier
from sklearn.experimental import enable_hist_gradient_boosting  # noqa # explicitly require this experimental feature
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from lightgbm import LGBMClassifier

import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier, Pool

import time # tic = time.perf_counter() ,toc = time.perf_counter()

import warnings;
warnings.filterwarnings('ignore');

    
#number of splits for cross validation
number_of_n_splits=10;

In [ ]:
#o fakelos poy periexei ta dataset kai einai ths morfhs class-data
dataDir =r"C:/Users/John/Desktop/machine learning/dataset/διπλωματικη/"
#dhmiourgia listas poy tha valoyme ta datasets
matlab_file = []
#vazomye ola ta arxeia poy periexontai sth dieuthnish 'dataDir' sth lista poy dhmioyrghsame
#epeidh einai arxeia ths matlab, xrhsimopoioyme th entolh read_mat gia th metatroph_
for file in os.listdir(dataDir) :
    matlab_file.append(read_mat(dataDir+file))

#save the names of datasets to only_names
only_names = [f for f in listdir("C:/Users/John/Desktop/machine learning/dataset/διπλωματικη/") if isfile(join("C:/Users/John/Desktop/machine learning/dataset/διπλωματικη/", f))]

In [ ]:
apotelesmata_df.to_excel('C:/Users/john/Desktop/machine learning/apotelesmata/f1_score.xlsx')  

In [ ]:
'''
#epeidh otan gyrizw ta apotelesmata apo to excel einai ola apothikeumena se string eftia3a funtion na ta gyrizei se float
apotelesmata_df_accuracy_score=pd.read_excel('C:/Users/John/Desktop/machine learning/apotelesmata/accuracy_score.xls')

apotelesmata_df_f1_score=pd.read_excel('C:/Users/John/Desktop/machine learning/apotelesmata/f1_score.xls') 

apotelesmata_df_specificity=pd.read_excel('C:/Users/John/Desktop/machine learning/apotelesmata/specificity.xls') 
'''

In [ ]:
def AdaBoost_function():
  y=AdaBoostClassifier()
  return y;

def Voting_function():
  y=VotingClassifier(estimators=[('BaggingClassifier', BaggingClassifier()), ('RandomForest', RandomForestClassifier()), ('ExtraTreesClassifier',ExtraTreesClassifier())], voting='hard')
  return y;

def Bagging_function():
  y=BaggingClassifier()
  return y;

def ExtraTrees_function():
  y=ExtraTreesClassifier()
  return y;

def RandomForest_function():
  y=RandomForestClassifier()
  return y;

def Stacking_function():
   estimators = [( RandomForestClassifier()),( ExtraTreesClassifier())]
   y = StackingClassifier(estimators=estimators, final_estimator=GaussianNB())
   return y;

def GradientBoosting_function():
  y=GradientBoostingClassifier()
  return y;

def LightGBM_function (data_train, class_train,data_test, parameters):
    d_train = lgb.Dataset(data_train, label= class_train)
    evals_result = {}
    model = lgb.train(parameters, d_train, 500, 
                      valid_sets=[d_train],
                      early_stopping_rounds=100, 
                       verbose_eval=50, 
                       evals_result=evals_result)
     
    predicted = model.predict(data_test, num_iteration=500)
    
    predicted_labels=np.argmax(predicted,axis=1);
    
    return predicted_labels

def LightGBM_function_proba (data_train, class_train,data_test, parameters):
    model = LGBMClassifier(objective='multiclass',metric = 'multi_logloss',num_class=class_num+1,n_estimators=500)
    model.fit(data_train, class_train)
    predicted_labels=model.predict_proba(data_test)
    
    return predicted_labels
      

def Catboost_function (data_train, class_train,data_test,class_test, parameters):
    model = CatBoostClassifier(**parameters)
    train_dataset = Pool(data=data_train,label=class_train)
    eval_dataset = Pool(data=data_test,label=class_test) 
    model.fit(train_dataset)
    predicted_labels = model.predict(eval_dataset)
    predicted_labels = np.reshape(predicted_labels, (1, len(predicted_labels)))
    predicted_labels=np.squeeze(predicted_labels)
    return predicted_labels

def Catboost_function_proba (data_train, class_train,data_test,class_test, parameters):
    model = CatBoostClassifier(**parameters)
    train_dataset = Pool(data=data_train,label=class_train)
    eval_dataset = Pool(data=data_test,label=class_test) 
    model.fit(train_dataset)
    predicted_labels = model.predict_proba(eval_dataset)
    return predicted_labels


def XGBoost_function (data_train, class_train,data_test,class_test, parameters):
   tr_data = xgb.DMatrix(data=data_train, label=class_train)
   dtest = xgb.DMatrix(data=data_test,label=class_test)
   watchlist = [(dtest, 'test'),(tr_data, 'train')]
   model_xgb = xgb.train(parameters, tr_data, 500,watchlist, early_stopping_rounds = 100, verbose_eval=50)
   predicted = model_xgb.predict(dtest,ntree_limit=model_xgb.best_ntree_limit)
   predicted_labels=np.argmax(predicted,axis=1);
   return predicted_labels

def XGBoost_function_proba (data_train, class_train,data_test,class_test, parameters):
    
   clf = xgb.XGBClassifier(n_estimators = 500, objective= 'multi:softprob')
   model_xgb=clf.fit(data_train, class_train,  early_stopping_rounds=100, eval_metric='mlogloss', eval_set=[(data_test, class_test)])
   predicted_labels = model_xgb.predict_proba(data_test)
   return predicted_labels

def string_to_float(a):
  a=a[:-1]
  a=a[1:]
  a=a.split(",")
  y=[float(i) for i in a]
  return y;

def ypologismos_specificity (m):
    if len(m)>0 :
      TP_temp=0;
      FP_temp=0;
      FN_temp=0;
      TN_temp=0;
      for i in range(0, len(m), 4):
          TP_temp=m[i]+TP_temp;
          FP_temp=m[i+1]+FP_temp;
          FN_temp=m[i+2]+FN_temp;
          TN_temp=m[i+3]+TN_temp; 
      TP=TP_temp;
      FP=FP_temp;
      FN=FN_temp;
      TN=TN_temp;
      #creating the confusion matrix
      #total_confusion_matrix=[[TP,FP],[FN,TN]]
      specificity_is=TN/(TN+FP)
    else:
      specificity_is=0
    
    return specificity_is


def voting(x,y,z) :      
        if (x == y) : 
            value=x
        elif (x == z) :
            value=x
        elif (y == z):
            value=y;
        else :
           value=y
        return value 

def Voting_G3(data_train, class_train,data_test,class_test, class_num, parameters_LighGBM,parameters_Catboost,parameters_XGBoost):

  #oi klaseis toy dataset
  classes=np.unique(class_df)
  #ypologismos pithanotitwn apo to kathe algorithmo
  predicted_LightGBM=LightGBM_function_proba (data_train, class_train,data_test, parameters_LightGBM)
  predicted_Catboost=Catboost_function_proba (data_train, class_train,data_test,class_test, parameters_Catboost)
  predicted_XGBoost=XGBoost_function_proba (data_train, class_train,data_test,class_test, parameters_XGBoost)
  
  
  samples=len(predicted_LightGBM/len(classes))
  predicted_labels=[0]*samples
    
  for j in range (0, len(predicted_LightGBM), len(classes)):
                   proba=[]
                   for h in range (len(classes)):
                      proba.append( predicted_LightGBM[j+h]+predicted_Catboost[j+h]+predicted_XGBoost[j+h])
                   #find the possition of the  biggest probability
                   predicted_labels[j]= classes[proba1.index(max_value)]
                    
  predicted_labels = np.reshape(predicted_labels, (1, len(predicted_labels)))
  predicted=np.squeeze(predicted_labels)                 

  return predicted;

In [ ]:
#dhmiourgia ndarray me theseis osoi kai oi classifiers poy tha xrhsimopoihsw apo th vivliothiki sklearn
clf=np.empty((7), dtype=object)

# Create adaboost classifer 
clf[0] = AdaBoost_function()

#Create Bagging Classifier 
clf[1] = Bagging_function()

#Create ExtraTreesClassifier
clf[2] = ExtraTrees_function()

#Create RandomForestClassifier
clf[3] = RandomForestClassifier()#RandomForest_function()

#Create StackingClassifier , Stack of estimators with a final classifier
#clf[4] = Stacking_function()
estimators = [( RandomForestClassifier()),( ExtraTreesClassifier())]
clf[4]= StackingClassifier(estimators=estimators, final_estimator=GaussianNB())

#Create VotingClassifier
clf[5] = Voting_function()

#Create GradientBoostingClassifier
clf[6] = GradientBoosting_function()

In [ ]:
for t in range (len(matlab_file)):
  data_df = pd.DataFrame(matlab_file[t]['data'])
  class_df = pd.DataFrame(matlab_file[t]['class'])
  
  
  # Creating an empty dictionary to store results
  apotelesmata = {}
  apotelesmata["key0"] = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]] 
  apotelesmata["key1"] = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]] 
  apotelesmata["key2"] = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
  apotelesmata["key3"] = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]] 
  apotelesmata["key4"] = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]] 
  apotelesmata["key5"] = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]] 
  apotelesmata["key6"] = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
  h=-1;
  print("to t einai",t, 'egine reset sto h',h) 
  for x,y in apotelesmata.items():
    apotelesmata[x];
    p=0;
    h=h+1;
    #choose classifier
    classifier=clf[h]
    print('Arxh, classifier',h , 'to x einai',x)
    for j in range (5):
       tic = time.perf_counter()
       kf = KFold(n_splits=number_of_n_splits, shuffle=True)
       #k-fold
       for train_index, test_index in kf.split(data_df,class_df):
          X_train, X_test = data_df.iloc[train_index], data_df.iloc[test_index]
          y_train, y_test = class_df.iloc[train_index], class_df.iloc[test_index]
          
          
          # Train Classifer
          
          model = classifier.fit(X_train, y_train)
          predicted = model.predict(X_test)
          
          #apothikeush sth lista apotelesmatwn
          apotelesmata[x][p+0].extend([predicted])
          
          #metatroph apo df se array
          y_test=y_test.to_numpy()
          #allagh diastasewn se 1-D
          y_test=np.ravel(y_test)
          
          #apothikeush sth lista apotelesmatwn
          apotelesmata[x][p+1].extend([y_test])
          
       toc = time.perf_counter()
       time_of=toc-tic
       #apothikeush sth lista apotelesmatwn
       apotelesmata[x][p+2].extend([time_of])
       p=p+3;
       print('~~teleiwse to K-fold noymero~~',j )
       print('xronos=',time_of)
    print("telos clf",h,"kai to t einai",t)
    with open(r'C:\Users\John\Desktop\machine learning\apotelesmata\duo_dataset\apotelesmata_' + str(only_names[t]) + '.p' , 'wb') as fp:
             pickle.dump(apotelesmata, fp, protocol=pickle.HIGHEST_PROTOCOL)  
  #save the data
  with open(r'C:\Users\John\Desktop\machine learning\apotelesmata\duo_dataset\apotelesmata_' + str(only_names[t]) + '.p' , 'wb') as fp:
    pickle.dump(apotelesmata, fp, protocol=pickle.HIGHEST_PROTOCOL)     

In [ ]:
## ~~ XGBoost  ~~

kf = KFold(n_splits=number_of_n_splits, shuffle=True)
#print('im in')


for t in range (len(matlab_file)):
  data_df = pd.DataFrame(matlab_file[t]['data'])
  class_df = pd.DataFrame(matlab_file[t]['class'])
  class_num=class_df[0].nunique()
  parameters = {'objective': 'multi:softprob', 
          'eval_metric': 'mlogloss',
          'num_class':class_num +1,
          'learning_rate': 0.1
          }
  
  # Creating an empty dictionary to store results
  apotelesmata_Xgboost = {}
  apotelesmata_Xgboost ["key0"] = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
  p=0;
  print('to arxeio einai to noymero',t )
  for j in range (5):
           tic = time.perf_counter()
    
#k-fold         
           for train_index, test_index in kf.split(data_df,class_df):
                     X_train, X_test = data_df.iloc[train_index], data_df.iloc[test_index]
                     y_train, y_test = class_df.iloc[train_index], class_df.iloc[test_index]
                     
                     
                     predicted=XGBoost_function (X_train, y_train,X_test,y_test, parameters)
                     
                     #apothikeush sth lista apotelesmatwn
                     apotelesmata_Xgboost ["key0"][p+0].extend([predicted])
          
                     #metatroph apo df se array
                     y_test=y_test.to_numpy()
                
                     #allagh diastasewn se 1-D
                     y_test=np.ravel(y_test)
          
                     #apothikeush sth lista apotelesmatwn
                     apotelesmata_Xgboost ["key0"][p+1].extend([y_test])
          
            
            
           toc = time.perf_counter()
           time_of=toc-tic
           #apothikeush sth lista apotelesmatwn
           apotelesmata_Xgboost ["key0"][p+2].extend([time_of])
           p=p+3;
           print('~~teleiwse to K-fold noymero~~',j )
           print('xronos=',time_of)
           #save the data
           with open(r'C:\Users\john\Desktop\machine learning\apotelesmata\duo_dataset\apotelesmata_XGBoost_' + str(only_names[t]) + '.p' , 'wb') as fp:
                    pickle.dump(apotelesmata_Xgboost, fp, protocol=pickle.HIGHEST_PROTOCOL) 
      


In [ ]:
## ~~    LightGBM   ~~

kf = KFold(n_splits=number_of_n_splits, shuffle=True)

#tre3imo gia kathe arxeio-dataset
for t in range (len(matlab_file)):
  data_df = pd.DataFrame(matlab_file[t]['data'])
  class_df = pd.DataFrame(matlab_file[t]['class'])
  #euresh twn monadikwn klasewn
  class_num=class_df[0].nunique()
  parameters = {
        'objective' : 'multiclass',
        'metric' : 'multi_logloss',
        'num_class':class_num+1,
        'learning_rate': 0.1
    }
  
  # Creating an empty dictionary to store results
  apotelesmata_LightGBM = {}
  apotelesmata_LightGBM ["key0"] = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
  p=0;
  print('to arxeio einai to noymero',t )
  for j in range (5):
           tic = time.perf_counter()
                
#k-fold         
           for train_index, test_index in kf.split(data_df,class_df):
                     
                     X_train, X_test = data_df.iloc[train_index], data_df.iloc[test_index]
                     y_train, y_test = class_df.iloc[train_index], class_df.iloc[test_index]
                     
                     y_test=y_test.to_numpy()
                     
                     predicted=LightGBM_function(X_train,y_train,y_test,parameters)
                                                
                     #apothikeush sth lista apotelesmatwn
                     apotelesmata_LightGBM ["key0"][p+0].extend([predicted])
          
                     #allagh diastasewn se 1-D
                     y_test=np.ravel(y_test)
          
                     #apothikeush sth lista apotelesmatwn
                     apotelesmata_LightGBM ["key0"][p+1].extend([y_test])
          
                     
            
            
           toc = time.perf_counter()
           #euresh xronou
           time_of=toc-tic
           #apothikeush sth lista apotelesmatwn
           apotelesmata_LightGBM ["key0"][p+2].extend([time_of])
           p=p+3;
           print('~~teleiwse to K-fold noymero~~',j )
           print('xronos=',time_of)
           #save the data
           with open(r'C:\Users\John\Desktop\machine learning\apotelesmata\duo_dataset\apotelesmata_LightGBM_' + str(only_names[t]) + '.p' , 'wb') as fp:
                    pickle.dump(apotelesmata_LightGBM, fp, protocol=pickle.HIGHEST_PROTOCOL) 
      


In [ ]:
## ~~   Catboost    ~~

kf = KFold(n_splits=number_of_n_splits,random_state=50, shuffle=True)
#print('im in')


for t in range (len(matlab_file)):
  data_df = pd.DataFrame(matlab_file[t]['data'])
  class_df = pd.DataFrame(matlab_file[t]['class'])
  parameters={'iterations':500,
             'verbose':50,
              'early_stopping_rounds':100,
              'loss_function':'MultiClass',
             'eval_metric':'MultiClass',
              'learning_rate': 0.1
             }

  
  # Creating an empty dictionary to store results
  apotelesmata_Catboost = {}
  apotelesmata_Catboost ["key0"] = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
  p=0;
  print('to arxeio einai to noymero',t )
  for j in range (5):
           tic = time.perf_counter()
                
#k-fold         
           for train_index, test_index in kf.split(data_df,class_df):
                     #print("Train:", train_index, "Validation:",test_index)
                     X_train, X_test = data_df.iloc[train_index], data_df.iloc[test_index]
                     y_train, y_test = class_df.iloc[train_index], class_df.iloc[test_index]
                     
                     y_test=y_test.to_numpy()
                     
                     predicted=Catboost_function (X_train, y_train,X_test,y_test, parameters)  
                     #apothikeush sth lista apotelesmatwn
                     predicted=np.ravel(predicted)
                     apotelesmata_Catboost ["key0"][p+0].extend([predicted])
          

                     #allagh diastasewn se 1-D
                     y_test=np.ravel(y_test)
          
                     #apothikeush sth lista apotelesmatwn
                     apotelesmata_Catboost ["key0"][p+1].extend([y_test])
          
            
            
           toc = time.perf_counter()
           time_of=toc-tic
           #apothikeush sth lista apotelesmatwn
           apotelesmata_Catboost ["key0"][p+2].extend([time_of])
           p=p+3;
           print('~~teleiwse to K-fold noymero~~',j )
           print('xronos=',time_of)
           #save the data
           with open(r'C:\Users\John\Desktop\machine learning\apotelesmata\duo_dataset\1apotelesmata_Catboost_' + str(only_names[t]) + '.p' , 'wb') as fp:
                    pickle.dump(apotelesmata_Catboost, fp, protocol=pickle.HIGHEST_PROTOCOL) 
      


In [ ]:
# for G3

kf = KFold(n_splits=number_of_n_splits, shuffle=True)
#print('im in')
  

for t in range (len(matlab_file)):

 
 data_df = pd.DataFrame(matlab_file[t]['data'])
 class_df = pd.DataFrame(matlab_file[t]['class'])
 class_num=class_df[0].nunique()
 
 parameters_LightGBM= {
        'objective' : 'multiclass',
        'metric' : 'multi_logloss',
        'num_class':class_num+1,
        'learning_rate': 0.1
    }
 parameters_Catboost={'iterations':500,
             'verbose':50,
              'early_stopping_rounds':100,
              'loss_function':'MultiClass',
               'learning_rate': 0.1
             }
 parameters_XGBoost= {'objective': 'multi:softprob', 
          'eval_metric': 'mlogloss',
          'num_class':class_num +1,
          'learning_rate': 0.1
          }




 # Creating an empty dictionary to store results
 apotelesmata_voting_nea = {}
 apotelesmata_voting_nea ["key0"] = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
 p=0;
 print('to arxeio einai to noymero',t )
 for j in range (5):
           tic = time.perf_counter()
    
#k-fold         
           for train_index, test_index in kf.split(data_df,class_df):
                     #print("Train:", train_index, "Validation:",test_index)
                     X_train, X_test = data_df.iloc[train_index], data_df.iloc[test_index]
                     y_train, y_test = class_df.iloc[train_index], class_df.iloc[test_index]
                     
                     
                     predicted=Voting_G3(X_train, y_train,X_test,y_test, class_num, parameters_LightGBM,parameters_Catboost,parameters_XGBoost)
                     #apothikeush sth lista apotelesmatwn
                     apotelesmata_voting_nea ["key0"][p+0].extend([predicted])
          
                     #metatroph apo df se array
                     y_test=y_test.to_numpy()

                     y_test=np.ravel(y_test)
          
                     #apothikeush sth lista apotelesmatwn
                     apotelesmata_voting_nea ["key0"][p+1].extend([y_test])
          
            
            
           toc = time.perf_counter()
           time_of=toc-tic
           print('o xronos poy ekane einai', time_of)
           #apothikeush sth lista apotelesmatwn
           apotelesmata_voting_nea ["key0"][p+2].extend([time_of])
           p=p+3;
           print('~~teleiwse to K-fold noymero~~',j )
           print('xronos=',time_of)
           #save the data
           with open(r'C:\Users\john\Desktop\machine learning\apotelesmata\duo_dataset\apotelesmata_voting_nea_' + str(only_names[t]) + '.p' , 'wb') as fp:
                    pickle.dump(apotelesmata_voting_nea, fp, protocol=pickle.HIGHEST_PROTOCOL) 
      


In [ ]:
#dhmiourgia dic poy tha fortwsoyme ta arxeia apo to fakelo
data_clf = {}
data_xgboost = {}
data_lightgbm = {}
data_catboost = {}
data_etoima_ola={}

#save the names of datasets to only_names
only_names_clf = [f for f in listdir("C:/Users/John/Desktop/machine learning/apotelesmata/clf/") if isfile(join("C:/Users/John/Desktop/machine learning/apotelesmata/clf/", f))]
only_names_xgboost = [f for f in listdir("C:/Users/John/Desktop/machine learning/apotelesmata/xgboost nea/") if isfile(join("C:/Users/John/Desktop/machine learning/apotelesmata/xgboost nea/", f))]
only_names_lightgbm = [f for f in listdir("C:/Users/John/Desktop/machine learning/apotelesmata/lightgbm nea/") if isfile(join("C:/Users/John/Desktop/machine learning/apotelesmata/lightgbm nea/", f))]
only_names_catboost = [f for f in listdir("C:/Users/John/Desktop/machine learning/apotelesmata/catboost nea/") if isfile(join("C:/Users/John/Desktop/machine learning/apotelesmata/catboost nea/", f))]
only_names_voting_nea = [f for f in listdir("C:/Users/John/Desktop/machine learning/apotelesmata/voting_nea/") if isfile(join("C:/Users/John/Desktop/machine learning/apotelesmata/voting_nea/", f))]

#load the data from folder
for i in range (26) :
    with open(r'C:/Users/John/Desktop/machine learning/apotelesmata/clf/'+str(only_names_clf[i]), 'rb') as fp:
     data_clf[str(only_names_clf[i])] = pickle.load(fp)
    with open(r'C:/Users/John/Desktop/machine learning/apotelesmata/xgboost nea/'+str(only_names_xgboost[i]), 'rb') as fp:
     data_xgboost[str(only_names_xgboost[i])] = pickle.load(fp)
    with open(r'C:/Users/John/Desktop/machine learning/apotelesmata/lightgbm nea/'+str(only_names_lightgbm[i]), 'rb') as fp:
     data_lightgbm[str(only_names_lightgbm[i])] = pickle.load(fp)
    with open(r'C:/Users/John/Desktop/machine learning/apotelesmata/catboost nea/'+str(only_names_catboost[i]), 'rb') as fp:
     data_catboost[str(only_names_catboost[i])] = pickle.load(fp)
    with open(r'C:/Users/John/Desktop/machine learning/apotelesmata/voting_nea/'+str(only_names_voting_nea[i]), 'rb') as fp:
     data_etoima_ola[str(only_names_voting_nea[i])] = pickle.load(fp)

In [ ]:
#mono gia to voting dokimh
etoima_xgboost={}
xronos_xgboost={}

etoima_lightgbm={}
xronos_lightgbm={}

etoima_catboost={}
xronos_catboost={}

etoima_clf_AdaBoost={}
xronos_clf_AdaBoost={}

etoima_clf_Bagging={}
xronos_clf_Bagging={}

etoima_clf_ExtraTrees={}
xronos_clf_ExtraTrees={}

etoima_clf_RandomForest={}
xronos_clf_RandomForest={}

etoima_clf_Stacking={}
xronos_clf_Stacking={}

etoima_clf_Voting={}
xronos_clf_Voting={}

etoima_clf_GradientBoosting={}
xronos_clf_GradientBoosting={}

etoima_ola={} #11
xronos_etoima_ola={} #11


for i in range (26) :
    x_xgboost=data_xgboost[str(only_names_xgboost[i])]['key0']
    
    x_lightgbm=data_lightgbm[str(only_names_lightgbm[i])]['key0']
    
    x_catboost=data_catboost[str(only_names_catboost[i])]['key0']
    
    x_etoima_ola=data_etoima_ola[str(only_names_voting_nea[i])]['key0'] #11 prosoxh exw valei mono 0
    
    
    x_clf_AdaBoost=data_clf[str(only_names_clf[i])]['key0']
    x_clf_Bagging=data_clf[str(only_names_clf[i])]['key1']
    x_clf_ExtraTrees=data_clf[str(only_names_clf[i])]['key2']
    x_clf_RandomForest=data_clf[str(only_names_clf[i])]['key3']
    x_clf_Stacking=data_clf[str(only_names_clf[i])]['key4']
    x_clf_Voting=data_clf[str(only_names_clf[i])]['key5']
    x_clf_GradientBoosting=data_clf[str(only_names_clf[i])]['key6']
    
    t=0
    
    merged_xgboost=[[],[],[],[],[]]
    merged1_xgboost=[[],[],[],[],[]]
    
    merged_clf=[[],[],[],[],[]]
    merged1_clf=[[],[],[],[],[]]
    
    merged_lightgbm=[[],[],[],[],[]]
    merged1_lightgbm=[[],[],[],[],[]]
    
    merged_catboost=[[],[],[],[],[]]
    merged1_catboost=[[],[],[],[],[]]
    
    merged_clf_AdaBoost=[[],[],[],[],[]]
    merged1_clf_AdaBoost=[[],[],[],[],[]]
    
    merged_clf_Bagging=[[],[],[],[],[]]
    merged1_clf_Bagging=[[],[],[],[],[]]
    
    merged_clf_ExtraTrees=[[],[],[],[],[]]
    merged1_clf_ExtraTrees=[[],[],[],[],[]]
    
    merged_clf_RandomForest=[[],[],[],[],[]]
    merged1_clf_RandomForest=[[],[],[],[],[]]
    
    merged_clf_Stacking=[[],[],[],[],[]]
    merged1_clf_Stacking=[[],[],[],[],[]]
    
    merged_clf_Voting=[[],[],[],[],[]]
    merged1_clf_Voting=[[],[],[],[],[]]
    
    merged_clf_GradientBoosting=[[],[],[],[],[]]
    merged1_clf_GradientBoosting=[[],[],[],[],[]]
    
    merged_etoima_ola=[[],[],[],[],[]]
    merged1_etoima_ola=[[],[],[],[],[]]
    
    for j in range (5) :
      merged_xgboost[j]= [item for sublist in x_xgboost[t] for item in sublist] #periexei ta predicted apotelesmata
      merged1_xgboost[j]= [item for sublist in x_xgboost[t+1] for item in sublist] #periexei ta true labels
        
      merged_lightgbm[j]= [item for sublist in x_lightgbm[t] for item in sublist] #periexei ta predicted apotelesmata
      merged1_lightgbm[j]= [item for sublist in x_lightgbm[t+1] for item in sublist] #periexei ta true labels
        
      merged_catboost[j]= [item for sublist in x_catboost[t] for item in sublist] #periexei ta predicted apotelesmata
      merged1_catboost[j]= [item for sublist in x_catboost[t+1] for item in sublist] #periexei ta true labels
        
      merged_clf_AdaBoost[j]= [item for sublist in x_clf_AdaBoost[t] for item in sublist] #periexei ta predicted apotelesmata
      merged1_clf_AdaBoost[j]= [item for sublist in x_clf_AdaBoost[t+1] for item in sublist] #periexei ta true labels

      merged_clf_Bagging[j]= [item for sublist in x_clf_Bagging[t] for item in sublist] #periexei ta predicted apotelesmata
      merged1_clf_Bagging[j]= [item for sublist in x_clf_Bagging[t+1] for item in sublist] #periexei ta true labels
        
      merged_clf_ExtraTrees[j]= [item for sublist in x_clf_ExtraTrees[t] for item in sublist] #periexei ta predicted apotelesmata
      merged1_clf_ExtraTrees[j]= [item for sublist in x_clf_ExtraTrees[t+1] for item in sublist] #periexei ta true labels        

      merged_clf_RandomForest[j]= [item for sublist in x_clf_RandomForest[t] for item in sublist] #periexei ta predicted apotelesmata
      merged1_clf_RandomForest[j]= [item for sublist in x_clf_RandomForest[t+1] for item in sublist] #periexei ta true labels  
        
      merged_clf_Stacking[j]= [item for sublist in x_clf_Stacking[t] for item in sublist] #periexei ta predicted apotelesmata
      merged1_clf_Stacking[j]= [item for sublist in x_clf_Stacking[t+1] for item in sublist] #periexei ta true labels 
        
      merged_clf_Voting[j]= [item for sublist in x_clf_Voting[t] for item in sublist] #periexei ta predicted apotelesmata
      merged1_clf_Voting[j]= [item for sublist in x_clf_Voting[t+1] for item in sublist] #periexei ta true labels
        
      merged_clf_GradientBoosting[j]= [item for sublist in x_clf_GradientBoosting[t] for item in sublist] #periexei ta predicted apotelesmata
      merged1_clf_GradientBoosting[j]= [item for sublist in x_clf_GradientBoosting[t+1] for item in sublist] #periexei ta true labels  
      
      merged_etoima_ola[j]=[item for sublist in x_etoima_ola[t] for item in sublist] #periexei ta predicted apotelesmata
      merged1_etoima_ola[j]=[item for sublist in x_etoima_ola[t+1] for item in sublist] #periexei ta true labels
      
        
      t=t+3
    #dhmiourgia dic gia apothikeysh twn merged listwn. prwta einai h lista merged, meta 3 medhinika kai meta h lista merged1
    etoima_xgboost[str(only_names_xgboost[i])]= [merged_xgboost,0,0,0,merged1_xgboost]
    xronos_xgboost[str(only_names_xgboost[i])]=[x_xgboost[2],x_xgboost[5],x_xgboost[8],x_xgboost[11],x_xgboost[14]]
    
    etoima_lightgbm[str(only_names_lightgbm[i])]= [merged_lightgbm,0,0,0,merged1_lightgbm]
    xronos_lightgbm[str(only_names_lightgbm[i])]=[x_lightgbm[2],x_lightgbm[5],x_lightgbm[8],x_lightgbm[11],x_lightgbm[14]]
    
    etoima_catboost[str(only_names_catboost[i])]= [merged_catboost,0,0,0,merged1_catboost] 
    xronos_catboost[str(only_names_catboost[i])]=[x_catboost[2],x_catboost[5],x_catboost[8],x_catboost[11],x_catboost[14]]
    
    etoima_clf_AdaBoost[str(only_names_clf[i])]= [merged_clf_AdaBoost,0,0,0,merged1_clf_AdaBoost]
    xronos_clf_AdaBoost[str(only_names_clf[i])]=[x_clf_AdaBoost[2],x_clf_AdaBoost[5],x_clf_AdaBoost[8],x_clf_AdaBoost[11],x_clf_AdaBoost[14]]
    
    etoima_clf_Bagging[str(only_names_clf[i])]= [merged_clf_Bagging,0,0,0,merged1_clf_Bagging]
    xronos_clf_Bagging[str(only_names_clf[i])]=[x_clf_Bagging[2],x_clf_Bagging[5],x_clf_Bagging[8],x_clf_Bagging[11],x_clf_Bagging[14]]
    
    etoima_clf_ExtraTrees[str(only_names_clf[i])]= [merged_clf_ExtraTrees,0,0,0,merged1_clf_ExtraTrees]
    xronos_clf_ExtraTrees[str(only_names_clf[i])]=[x_clf_ExtraTrees[2],x_clf_ExtraTrees[5],x_clf_ExtraTrees[8],x_clf_ExtraTrees[11],x_clf_ExtraTrees[14]]
    
    etoima_clf_RandomForest[str(only_names_clf[i])]= [merged_clf_RandomForest,0,0,0,merged1_clf_RandomForest]
    xronos_clf_RandomForest[str(only_names_clf[i])]=[x_clf_RandomForest[2],x_clf_RandomForest[5],x_clf_RandomForest[8],x_clf_RandomForest[11],x_clf_RandomForest[14]]
    
    etoima_clf_Stacking[str(only_names_clf[i])]= [merged_clf_Stacking,0,0,0,merged1_clf_Stacking]
    xronos_clf_Stacking[str(only_names_clf[i])]=[x_clf_Stacking[2],x_clf_Stacking[5],x_clf_Stacking[8],x_clf_Stacking[11],x_clf_Stacking[14]]
    
    etoima_clf_Voting[str(only_names_clf[i])]= [merged_clf_Voting,0,0,0,merged1_clf_Voting]
    xronos_clf_Voting[str(only_names_clf[i])]=[x_clf_Voting[2],x_clf_Voting[5],x_clf_Voting[8],x_clf_Voting[11],x_clf_Voting[14]]
    
    etoima_clf_GradientBoosting[str(only_names_clf[i])]= [merged_clf_GradientBoosting,0,0,0,merged1_clf_GradientBoosting]
    xronos_clf_GradientBoosting[str(only_names_clf[i])]=[x_clf_GradientBoosting[2],x_clf_GradientBoosting[5],x_clf_GradientBoosting[8],x_clf_GradientBoosting[11],x_clf_GradientBoosting[14]]
    
    etoima_ola[str(only_names_voting_nea[i])]=[merged_etoima_ola,0,0,0,merged1_etoima_ola] #prosoxh sto pointer, exw valei 0
    

In [ ]:
#ypologismos xronou mean-std
xronos_mean_std_xgboost={}
xronos_mean_std_catboost={}
xronos_mean_std_AdaBoost={}
xronos_mean_std_Bagging={}
xronos_mean_std_ExtraTrees={}
xronos_mean_std_RandomForest={}
xronos_mean_std_Stacking={}
xronos_mean_std_Voting={}
xronos_mean_std_GradientBoosting={}


for i in range (26) :
    xronos_mean_std_xgboost[str(only_names_xgboost[i])]=[np.mean(xronos_xgboost[str(only_names_xgboost[i])]),np.std(xronos_xgboost[str(only_names_xgboost[i])])]
    
    xronos_mean_std_lightgbm[str(only_names_lightgbm[i])]=[np.mean(xronos_lightgbm[str(only_names_lightgbm[i])]),np.std(xronos_lightgbm[str(only_names_lightgbm[i])])]

    xronos_mean_std_catboost[str(only_names_catboost[i])]=[np.mean(xronos_catboost[str(only_names_catboost[i])]),np.std(xronos_catboost[str(only_names_catboost[i])])]
    
    xronos_mean_std_AdaBoost[str(only_names_clf[i])]=[np.mean(xronos_clf_AdaBoost[str(only_names_clf[i])]),np.std(xronos_clf_AdaBoost[str(only_names_clf[i])])]
    
    xronos_mean_std_Bagging[str(only_names_clf[i])]=[np.mean(xronos_clf_Bagging[str(only_names_clf[i])]),np.std(xronos_clf_Bagging[str(only_names_clf[i])])]
    
    xronos_mean_std_ExtraTrees[str(only_names_clf[i])]=[np.mean(xronos_clf_ExtraTrees[str(only_names_clf[i])]),np.std(xronos_clf_ExtraTrees[str(only_names_clf[i])])]
    
    xronos_mean_std_RandomForest[str(only_names_clf[i])]=[np.mean(xronos_clf_RandomForest[str(only_names_clf[i])]),np.std(xronos_clf_RandomForest[str(only_names_clf[i])])]
    
    xronos_mean_std_Stacking[str(only_names_clf[i])]=[np.mean(xronos_clf_Stacking[str(only_names_clf[i])]),np.std(xronos_clf_Stacking[str(only_names_clf[i])])]
    
    xronos_mean_std_Voting[str(only_names_clf[i])]=[np.mean(xronos_clf_Voting[str(only_names_clf[i])]),np.std(xronos_clf_Voting[str(only_names_clf[i])])]
    
    xronos_mean_std_GradientBoosting[str(only_names_clf[i])]=[np.mean(xronos_clf_GradientBoosting[str(only_names_clf[i])]),np.std(xronos_clf_GradientBoosting[str(only_names_clf[i])])]

In [ ]:
#set the metric  
metric=accuracy_score  #  ~f1_score,accuracy_score,specifity --> alla3e kai to fakelo pou apothikeueis

apotelesmata_ola={}
apotelesmata_XGBoost={}
apotelesmata_lightgbm={}
apotelesmata_catboost={}
apotelesmata_AdaBoost={}
apotelesmata_Bagging={}
apotelesmata_ExtraTrees={}
apotelesmata_RandomForest={}
apotelesmata_Stacking={}
apotelesmata_Voting={}
apotelesmata_GradientBoosting={}

apotelesmata_ana_dataset={}

for i in range (26):
    
    a= metric(etoima_xgboost[str(only_names_xgboost[i])][0][0],etoima_xgboost[str(only_names_xgboost[i])][4][0])
    a1= metric(etoima_xgboost[str(only_names_xgboost[i])][0][1],etoima_xgboost[str(only_names_xgboost[i])][4][1])
    a2= metric(etoima_xgboost[str(only_names_xgboost[i])][0][2],etoima_xgboost[str(only_names_xgboost[i])][4][2])
    a3= metric(etoima_xgboost[str(only_names_xgboost[i])][0][3],etoima_xgboost[str(only_names_xgboost[i])][4][3])
    a4= metric(etoima_xgboost[str(only_names_xgboost[i])][0][4],etoima_xgboost[str(only_names_xgboost[i])][4][4])

    b= metric(etoima_lightgbm[str(only_names_lightgbm[i])][0][0],etoima_lightgbm[str(only_names_lightgbm[i])][4][0])
    b1= metric(etoima_lightgbm[str(only_names_lightgbm[i])][0][1],etoima_lightgbm[str(only_names_lightgbm[i])][4][1])
    b2= metric(etoima_lightgbm[str(only_names_lightgbm[i])][0][2],etoima_lightgbm[str(only_names_lightgbm[i])][4][2])
    b3= metric(etoima_lightgbm[str(only_names_lightgbm[i])][0][3],etoima_lightgbm[str(only_names_lightgbm[i])][4][3])
    b4= metric(etoima_lightgbm[str(only_names_lightgbm[i])][0][4],etoima_lightgbm[str(only_names_lightgbm[i])][4][4])

    c= metric(etoima_catboost[str(only_names_catboost[i])][0][0],etoima_catboost[str(only_names_catboost[i])][4][0])
    c1= metric(etoima_catboost[str(only_names_catboost[i])][0][1],etoima_catboost[str(only_names_catboost[i])][4][1])
    c2= metric(etoima_catboost[str(only_names_catboost[i])][0][2],etoima_catboost[str(only_names_catboost[i])][4][2])
    c3= metric(etoima_catboost[str(only_names_catboost[i])][0][3],etoima_catboost[str(only_names_catboost[i])][4][3])
    c4= metric(etoima_catboost[str(only_names_catboost[i])][0][4],etoima_catboost[str(only_names_catboost[i])][4][4])

    d= metric(etoima_clf_AdaBoost[str(only_names_clf[i])][0][0],etoima_clf_AdaBoost[str(only_names_clf[i])][4][0])
    d1= metric(etoima_clf_AdaBoost[str(only_names_clf[i])][0][1],etoima_clf_AdaBoost[str(only_names_clf[i])][4][1])
    d2= metric(etoima_clf_AdaBoost[str(only_names_clf[i])][0][2],etoima_clf_AdaBoost[str(only_names_clf[i])][4][2])
    d3= metric(etoima_clf_AdaBoost[str(only_names_clf[i])][0][3],etoima_clf_AdaBoost[str(only_names_clf[i])][4][3])
    d4= metric(etoima_clf_AdaBoost[str(only_names_clf[i])][0][4],etoima_clf_AdaBoost[str(only_names_clf[i])][4][4])

    e= metric(etoima_clf_Bagging[str(only_names_clf[i])][0][0],etoima_clf_Bagging[str(only_names_clf[i])][4][0])
    e1= metric(etoima_clf_Bagging[str(only_names_clf[i])][0][1],etoima_clf_Bagging[str(only_names_clf[i])][4][1])
    e2= metric(etoima_clf_Bagging[str(only_names_clf[i])][0][2],etoima_clf_Bagging[str(only_names_clf[i])][4][2])
    e3= metric(etoima_clf_Bagging[str(only_names_clf[i])][0][3],etoima_clf_Bagging[str(only_names_clf[i])][4][3])
    e4= metric(etoima_clf_Bagging[str(only_names_clf[i])][0][4],etoima_clf_Bagging[str(only_names_clf[i])][4][4])

    f= metric(etoima_clf_ExtraTrees[str(only_names_clf[i])][0][0],etoima_clf_ExtraTrees[str(only_names_clf[i])][4][0])
    f1= metric(etoima_clf_ExtraTrees[str(only_names_clf[i])][0][1],etoima_clf_ExtraTrees[str(only_names_clf[i])][4][1])
    f2= metric(etoima_clf_ExtraTrees[str(only_names_clf[i])][0][2],etoima_clf_ExtraTrees[str(only_names_clf[i])][4][2])
    f3= metric(etoima_clf_ExtraTrees[str(only_names_clf[i])][0][3],etoima_clf_ExtraTrees[str(only_names_clf[i])][4][3])
    f4= metric(etoima_clf_ExtraTrees[str(only_names_clf[i])][0][4],etoima_clf_ExtraTrees[str(only_names_clf[i])][4][4])

    g= metric(etoima_clf_RandomForest[str(only_names_clf[i])][0][0],etoima_clf_RandomForest[str(only_names_clf[i])][4][0])
    g1= metric(etoima_clf_RandomForest[str(only_names_clf[i])][0][1],etoima_clf_RandomForest[str(only_names_clf[i])][4][1])
    g2= metric(etoima_clf_RandomForest[str(only_names_clf[i])][0][2],etoima_clf_RandomForest[str(only_names_clf[i])][4][2])
    g3= metric(etoima_clf_RandomForest[str(only_names_clf[i])][0][3],etoima_clf_RandomForest[str(only_names_clf[i])][4][3])
    g4= metric(etoima_clf_RandomForest[str(only_names_clf[i])][0][4],etoima_clf_RandomForest[str(only_names_clf[i])][4][4])

    h= metric(etoima_clf_Stacking[str(only_names_clf[i])][0][0],etoima_clf_Stacking[str(only_names_clf[i])][4][0])
    h1= metric(etoima_clf_Stacking[str(only_names_clf[i])][0][1],etoima_clf_Stacking[str(only_names_clf[i])][4][1])
    h2= metric(etoima_clf_Stacking[str(only_names_clf[i])][0][2],etoima_clf_Stacking[str(only_names_clf[i])][4][2])
    h3= metric(etoima_clf_Stacking[str(only_names_clf[i])][0][3],etoima_clf_Stacking[str(only_names_clf[i])][4][3])
    h4= metric(etoima_clf_Stacking[str(only_names_clf[i])][0][4],etoima_clf_Stacking[str(only_names_clf[i])][4][4])

    m= metric(etoima_clf_Voting[str(only_names_clf[i])][0][0],etoima_clf_Voting[str(only_names_clf[i])][4][0])
    m1= metric(etoima_clf_Voting[str(only_names_clf[i])][0][1],etoima_clf_Voting[str(only_names_clf[i])][4][1])
    m2= metric(etoima_clf_Voting[str(only_names_clf[i])][0][2],etoima_clf_Voting[str(only_names_clf[i])][4][2])
    m3= metric(etoima_clf_Voting[str(only_names_clf[i])][0][3],etoima_clf_Voting[str(only_names_clf[i])][4][3])
    m4= metric(etoima_clf_Voting[str(only_names_clf[i])][0][4],etoima_clf_Voting[str(only_names_clf[i])][4][4])

    j= metric(etoima_clf_GradientBoosting[str(only_names_clf[i])][0][0],etoima_clf_GradientBoosting[str(only_names_clf[i])][4][0])
    j1= metric(etoima_clf_GradientBoosting[str(only_names_clf[i])][0][1],etoima_clf_GradientBoosting[str(only_names_clf[i])][4][1])
    j2= metric(etoima_clf_GradientBoosting[str(only_names_clf[i])][0][2],etoima_clf_GradientBoosting[str(only_names_clf[i])][4][2])
    j3= metric(etoima_clf_GradientBoosting[str(only_names_clf[i])][0][3],etoima_clf_GradientBoosting[str(only_names_clf[i])][4][3])
    j4= metric(etoima_clf_GradientBoosting[str(only_names_clf[i])][0][4],etoima_clf_GradientBoosting[str(only_names_clf[i])][4][4])

  #exw peira3ei to pointer, na douleuei mono gia prwto arxeio  
    l= metric(etoima_ola[str(only_names_voting_nea[i])][0][0],etoima_ola[str(only_names_voting_nea[i])][4][0])
    l1= metric(etoima_ola[str(only_names_voting_nea[i])][0][1],etoima_ola[str(only_names_voting_nea[i])][4][1])
    l2= metric(etoima_ola[str(only_names_voting_nea[i])][0][2],etoima_ola[str(only_names_voting_nea[i])][4][2])
    l3= metric(etoima_ola[str(only_names_voting_nea[i])][0][3],etoima_ola[str(only_names_voting_nea[i])][4][3])
    l4= metric(etoima_ola[str(only_names_voting_nea[i])][0][4],etoima_ola[str(only_names_voting_nea[i])][4][4])
    
    
    
    
    
    
    apotelesmata_XGBoost[str(only_names_xgboost[i])]=[a,a1,a2,a3,a4]
    
    apotelesmata_lightgbm[str(only_names_lightgbm[i])]=[b,b1,b2,b3,b4]
    
    apotelesmata_catboost[str(only_names_catboost[i])]=[c,c1,c2,c3,c4]
    
    apotelesmata_AdaBoost[str(only_names_clf[i])]=[d,d1,d2,d3,d4]
    
    apotelesmata_Bagging[str(only_names_clf[i])]=[e,e1,e2,e3,e4]
    
    apotelesmata_ExtraTrees[str(only_names_clf[i])]=[f,f1,f2,f3,f4]
    
    apotelesmata_RandomForest[str(only_names_clf[i])]=[g,g1,g2,g3,g4]
    
    apotelesmata_Stacking[str(only_names_clf[i])]=[h,h1,h2,h3,h4]
    
    apotelesmata_Voting[str(only_names_clf[i])]=[m,m1,m2,m3,m4]
    
    apotelesmata_GradientBoosting[str(only_names_clf[i])]=[j,j1,j2,j3,j4]
    
    apotelesmata_ola[str(only_names_voting_nea[i])]=[l,l1,l2,l3,l4] 
    
    apotelesmata_ana_dataset[str(only_names_clf[i])]= [apotelesmata_ola[str(only_names_voting_nea[i])],apotelesmata_XGBoost[str(only_names_xgboost[i])],apotelesmata_lightgbm[str(only_names_lightgbm[i])],apotelesmata_catboost[str(only_names_catboost[i])],apotelesmata_AdaBoost[str(only_names_clf[i])],apotelesmata_Bagging[str(only_names_clf[i])],apotelesmata_ExtraTrees[str(only_names_clf[i])],apotelesmata_RandomForest[str(only_names_clf[i])],apotelesmata_Stacking[str(only_names_clf[i])],apotelesmata_Voting[str(only_names_clf[i])],apotelesmata_GradientBoosting[str(only_names_clf[i])]]

    
    
apotelesmata_df=pd.DataFrame.from_dict(apotelesmata_ana_dataset)

apotelesmata_df.to_excel('C:/Users/john/Desktop/machine learning/apotelesmata/dokimh_excel/accuracy_score_work.xls')

In [ ]:
#ypologismos specificity


apotelesmata_XGBoost={}
apotelesmata_lightgbm={}
apotelesmata_catboost={}
apotelesmata_AdaBoost={}
apotelesmata_Bagging={}
apotelesmata_ExtraTrees={}
apotelesmata_RandomForest={}
apotelesmata_Stacking={}
apotelesmata_Voting={}
apotelesmata_GradientBoosting={}

apotelesmata_ana_dataset={}

for i in range (26):
    a= ypologismos_specificity(multilabel_confusion_matrix(etoima_xgboost[str(only_names_xgboost[i])][0][0],etoima_xgboost[str(only_names_xgboost[i])][4][0]).ravel())
    a1= ypologismos_specificity(multilabel_confusion_matrix(etoima_xgboost[str(only_names_xgboost[i])][0][1],etoima_xgboost[str(only_names_xgboost[i])][4][1]).ravel())
    a2= ypologismos_specificity(multilabel_confusion_matrix(etoima_xgboost[str(only_names_xgboost[i])][0][2],etoima_xgboost[str(only_names_xgboost[i])][4][2]).ravel())
    a3= ypologismos_specificity(multilabel_confusion_matrix(etoima_xgboost[str(only_names_xgboost[i])][0][3],etoima_xgboost[str(only_names_xgboost[i])][4][3]).ravel())
    a4= ypologismos_specificity(multilabel_confusion_matrix(etoima_xgboost[str(only_names_xgboost[i])][0][4],etoima_xgboost[str(only_names_xgboost[i])][4][4]).ravel())

    b= ypologismos_specificity(multilabel_confusion_matrix(etoima_lightgbm[str(only_names_lightgbm[i])][0][0],etoima_lightgbm[str(only_names_lightgbm[i])][4][0]).ravel())
    b1= ypologismos_specificity(multilabel_confusion_matrix(etoima_lightgbm[str(only_names_lightgbm[i])][0][1],etoima_lightgbm[str(only_names_lightgbm[i])][4][1]).ravel())
    b2= ypologismos_specificity(multilabel_confusion_matrix(etoima_lightgbm[str(only_names_lightgbm[i])][0][2],etoima_lightgbm[str(only_names_lightgbm[i])][4][2]).ravel())
    b3= ypologismos_specificity(multilabel_confusion_matrix(etoima_lightgbm[str(only_names_lightgbm[i])][0][3],etoima_lightgbm[str(only_names_lightgbm[i])][4][3]).ravel())
    b4= ypologismos_specificity(multilabel_confusion_matrix(etoima_lightgbm[str(only_names_lightgbm[i])][0][4],etoima_lightgbm[str(only_names_lightgbm[i])][4][4]).ravel())

    c= ypologismos_specificity(multilabel_confusion_matrix(etoima_catboost[str(only_names_catboost[i])][0][0],etoima_catboost[str(only_names_catboost[i])][4][0]).ravel())
    c1= ypologismos_specificity(multilabel_confusion_matrix(etoima_catboost[str(only_names_catboost[i])][0][1],etoima_catboost[str(only_names_catboost[i])][4][1]).ravel())
    c2= ypologismos_specificity(multilabel_confusion_matrix(etoima_catboost[str(only_names_catboost[i])][0][2],etoima_catboost[str(only_names_catboost[i])][4][2]).ravel())
    c3= ypologismos_specificity(multilabel_confusion_matrix(etoima_catboost[str(only_names_catboost[i])][0][3],etoima_catboost[str(only_names_catboost[i])][4][3]).ravel())
    c4= ypologismos_specificity(multilabel_confusion_matrix(etoima_catboost[str(only_names_catboost[i])][0][4],etoima_catboost[str(only_names_catboost[i])][4][4]).ravel())

    d= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_AdaBoost[str(only_names_clf[i])][0][0],etoima_clf_AdaBoost[str(only_names_clf[i])][4][0]).ravel())
    d1= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_AdaBoost[str(only_names_clf[i])][0][1],etoima_clf_AdaBoost[str(only_names_clf[i])][4][1]).ravel())
    d2= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_AdaBoost[str(only_names_clf[i])][0][2],etoima_clf_AdaBoost[str(only_names_clf[i])][4][2]).ravel())
    d3= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_AdaBoost[str(only_names_clf[i])][0][3],etoima_clf_AdaBoost[str(only_names_clf[i])][4][3]).ravel())
    d4= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_AdaBoost[str(only_names_clf[i])][0][4],etoima_clf_AdaBoost[str(only_names_clf[i])][4][4]).ravel())

    e= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_Bagging[str(only_names_clf[i])][0][0],etoima_clf_Bagging[str(only_names_clf[i])][4][0]).ravel())
    e1= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_Bagging[str(only_names_clf[i])][0][1],etoima_clf_Bagging[str(only_names_clf[i])][4][1]).ravel())
    e2= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_Bagging[str(only_names_clf[i])][0][2],etoima_clf_Bagging[str(only_names_clf[i])][4][2]).ravel())
    e3= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_Bagging[str(only_names_clf[i])][0][3],etoima_clf_Bagging[str(only_names_clf[i])][4][3]).ravel())
    e4= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_Bagging[str(only_names_clf[i])][0][4],etoima_clf_Bagging[str(only_names_clf[i])][4][4]).ravel())

    f= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_ExtraTrees[str(only_names_clf[i])][0][0],etoima_clf_ExtraTrees[str(only_names_clf[i])][4][0]).ravel())
    f1= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_ExtraTrees[str(only_names_clf[i])][0][1],etoima_clf_ExtraTrees[str(only_names_clf[i])][4][1]).ravel())
    f2= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_ExtraTrees[str(only_names_clf[i])][0][2],etoima_clf_ExtraTrees[str(only_names_clf[i])][4][2]).ravel())
    f3= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_ExtraTrees[str(only_names_clf[i])][0][3],etoima_clf_ExtraTrees[str(only_names_clf[i])][4][3]).ravel())
    f4= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_ExtraTrees[str(only_names_clf[i])][0][4],etoima_clf_ExtraTrees[str(only_names_clf[i])][4][4]).ravel())

    g= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_RandomForest[str(only_names_clf[i])][0][0],etoima_clf_RandomForest[str(only_names_clf[i])][4][0]).ravel())
    g1= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_RandomForest[str(only_names_clf[i])][0][1],etoima_clf_RandomForest[str(only_names_clf[i])][4][1]).ravel())
    g2= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_RandomForest[str(only_names_clf[i])][0][2],etoima_clf_RandomForest[str(only_names_clf[i])][4][2]).ravel())
    g3= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_RandomForest[str(only_names_clf[i])][0][3],etoima_clf_RandomForest[str(only_names_clf[i])][4][3]).ravel())
    g4= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_RandomForest[str(only_names_clf[i])][0][4],etoima_clf_RandomForest[str(only_names_clf[i])][4][4]).ravel())

    h= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_Stacking[str(only_names_clf[i])][0][0],etoima_clf_Stacking[str(only_names_clf[i])][4][0]).ravel())
    h1= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_Stacking[str(only_names_clf[i])][0][1],etoima_clf_Stacking[str(only_names_clf[i])][4][1]).ravel())
    h2= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_Stacking[str(only_names_clf[i])][0][2],etoima_clf_Stacking[str(only_names_clf[i])][4][2]).ravel())
    h3= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_Stacking[str(only_names_clf[i])][0][3],etoima_clf_Stacking[str(only_names_clf[i])][4][3]).ravel())
    h4= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_Stacking[str(only_names_clf[i])][0][4],etoima_clf_Stacking[str(only_names_clf[i])][4][4]).ravel())

    m= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_Voting[str(only_names_clf[i])][0][0],etoima_clf_Voting[str(only_names_clf[i])][4][0]).ravel())
    m1= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_Voting[str(only_names_clf[i])][0][1],etoima_clf_Voting[str(only_names_clf[i])][4][1]).ravel())
    m2= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_Voting[str(only_names_clf[i])][0][2],etoima_clf_Voting[str(only_names_clf[i])][4][2]).ravel())
    m3= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_Voting[str(only_names_clf[i])][0][3],etoima_clf_Voting[str(only_names_clf[i])][4][3]).ravel())
    m4= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_Voting[str(only_names_clf[i])][0][4],etoima_clf_Voting[str(only_names_clf[i])][4][4]).ravel())

    j= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_GradientBoosting[str(only_names_clf[i])][0][0],etoima_clf_GradientBoosting[str(only_names_clf[i])][4][0]).ravel())
    j1= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_GradientBoosting[str(only_names_clf[i])][0][1],etoima_clf_GradientBoosting[str(only_names_clf[i])][4][1]).ravel())
    j2= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_GradientBoosting[str(only_names_clf[i])][0][2],etoima_clf_GradientBoosting[str(only_names_clf[i])][4][2]).ravel())
    j3= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_GradientBoosting[str(only_names_clf[i])][0][3],etoima_clf_GradientBoosting[str(only_names_clf[i])][4][3]).ravel())
    j4= ypologismos_specificity(multilabel_confusion_matrix(etoima_clf_GradientBoosting[str(only_names_clf[i])][0][4],etoima_clf_GradientBoosting[str(only_names_clf[i])][4][4]).ravel())

    

    l= ypologismos_specificity(multilabel_confusion_matrix(etoima_ola[str(only_names_voting_nea[i])][0][0],etoima_ola[str(only_names_voting_nea[i])][4][0]).ravel())
    l1= ypologismos_specificity(multilabel_confusion_matrix(etoima_ola[str(only_names_voting_nea[i])][0][1],etoima_ola[str(only_names_voting_nea[i])][4][1]).ravel())
    l2= ypologismos_specificity(multilabel_confusion_matrix(etoima_ola[str(only_names_voting_nea[i])][0][2],etoima_ola[str(only_names_voting_nea[i])][4][2]).ravel())
    l3= ypologismos_specificity(multilabel_confusion_matrix(etoima_ola[str(only_names_voting_nea[i])][0][3],etoima_ola[str(only_names_voting_nea[i])][4][3]).ravel())
    l4= ypologismos_specificity(multilabel_confusion_matrix(etoima_ola[str(only_names_voting_nea[i])][0][4],etoima_ola[str(only_names_voting_nea[i])][4][4]).ravel())
    
    
    apotelesmata_XGBoost[str(only_names_xgboost[i])]=[a,a1,a2,a3,a4]
    
    apotelesmata_lightgbm[str(only_names_lightgbm[i])]=[b,b1,b2,b3,b4]
    
    apotelesmata_catboost[str(only_names_catboost[i])]=[c,c1,c2,c3,c4]
    
    apotelesmata_AdaBoost[str(only_names_clf[i])]=[d,d1,d2,d3,d4]
    
    apotelesmata_Bagging[str(only_names_clf[i])]=[e,e1,e2,e3,e4]
    
    apotelesmata_ExtraTrees[str(only_names_clf[i])]=[f,f1,f2,f3,f4]
    
    apotelesmata_RandomForest[str(only_names_clf[i])]=[g,g1,g2,g3,g4]
    
    apotelesmata_Stacking[str(only_names_clf[i])]=[h,h1,h2,h3,h4]
    
    apotelesmata_Voting[str(only_names_clf[i])]=[m,m1,m2,m3,m4]
    
    apotelesmata_GradientBoosting[str(only_names_clf[i])]=[j,j1,j2,j3,j4]
    
    apotelesmata_ola[str(only_names_voting_nea[i])]=[l,l1,l2,l3,l4]
    
    apotelesmata_ana_dataset[str(only_names_clf[i])]= [apotelesmata_ola[str(only_names_voting_nea[i])],apotelesmata_XGBoost[str(only_names_xgboost[i])],apotelesmata_lightgbm[str(only_names_lightgbm[i])],apotelesmata_catboost[str(only_names_catboost[i])],apotelesmata_AdaBoost[str(only_names_clf[i])],apotelesmata_Bagging[str(only_names_clf[i])],apotelesmata_ExtraTrees[str(only_names_clf[i])],apotelesmata_RandomForest[str(only_names_clf[i])],apotelesmata_Stacking[str(only_names_clf[i])],apotelesmata_Voting[str(only_names_clf[i])],apotelesmata_GradientBoosting[str(only_names_clf[i])]]

apotelesmata_df=pd.DataFrame.from_dict(apotelesmata_ana_dataset)

apotelesmata_df.to_excel('C:/Users/john/Desktop/machine learning/apotelesmata/specificity_work.xls')  

In [ ]:
#epeidh otan gyrizw ta apotelesmata apo to excel einai ola apothikeumena se string eftia3a funtion na ta gyrizei se float

apotelesmata_df_accuracy_score=pd.read_excel('C:/Users/John/Desktop/pros telos apo 14.06/apotelesmata/accuracy_score.xls')

apotelesmata_df_f1_score=pd.read_excel('C:/Users/John/Desktop/pros telos apo 14.06/apotelesmata/f1_score.xls') 

apotelesmata_df_specificity=pd.read_excel('C:/Users/John/Desktop/pros telos apo 14.06/apotelesmata/specificity.xls') 

In [ ]:
#allagh onomatos sto only_names
only_names_clf_new=[0]*26
for i in range(26):
    only_names_clf_new[i]=only_names_clf[i][13:]
    only_names_clf_new[i]=only_names_clf_new[i][:-6]

In [ ]:
#dhmiourgia boxplot
for i in range (26):
 d_1 = {'G3': string_to_float(apotelesmata_df_accuracy_score[str(only_names_clf[i])][0]),'XGBoost': string_to_float(apotelesmata_df_accuracy_score[str(only_names_clf[i])][1]), 'LightGBM': string_to_float(apotelesmata_df_accuracy_score[str(only_names_clf[i])][2]),'Catboost': string_to_float(apotelesmata_df_accuracy_score[str(only_names_clf[i])][3]), 'AdaBoost': string_to_float(apotelesmata_df_accuracy_score[str(only_names_clf[i])][4]), 'Bagging': string_to_float(apotelesmata_df_accuracy_score[str(only_names_clf[i])][5]), 'ExtraTrees': string_to_float(apotelesmata_df_accuracy_score[str(only_names_clf[i])][6]), 'RandomForest': string_to_float(apotelesmata_df_accuracy_score[str(only_names_clf[i])][7]), 'Stacking': string_to_float(apotelesmata_df_accuracy_score[str(only_names_clf[i])][8]), 'Voting': string_to_float(apotelesmata_df_accuracy_score[str(only_names_clf[i])][9]), 'GradientBoosting': string_to_float(apotelesmata_df_accuracy_score[str(only_names_clf[i])][10])}
 d_2= {'G3': string_to_float(apotelesmata_df_f1_score[str(only_names_clf[i])][0]),'XGBoost': string_to_float(apotelesmata_df_f1_score[str(only_names_clf[i])][1]), 'LightGBM': string_to_float(apotelesmata_df_f1_score[str(only_names_clf[i])][2]),'Catboost': string_to_float(apotelesmata_df_f1_score[str(only_names_clf[i])][3]), 'AdaBoost': string_to_float(apotelesmata_df_f1_score[str(only_names_clf[i])][4]), 'Bagging': string_to_float(apotelesmata_df_f1_score[str(only_names_clf[i])][5]), 'ExtraTrees': string_to_float(apotelesmata_df_f1_score[str(only_names_clf[i])][6]), 'RandomForest': string_to_float(apotelesmata_df_f1_score[str(only_names_clf[i])][6]), 'Stacking': string_to_float(apotelesmata_df_f1_score[str(only_names_clf[i])][8]), 'Voting': string_to_float(apotelesmata_df_f1_score[str(only_names_clf[i])][9]), 'GradientBoosting': string_to_float(apotelesmata_df_f1_score[str(only_names_clf[i])][10])}
 d_3={'G3': string_to_float(apotelesmata_df_specificity[str(only_names_clf[i])][0]),'XGBoost': string_to_float(apotelesmata_df_specificity[str(only_names_clf[i])][1]), 'LightGBM': string_to_float(apotelesmata_df_specificity[str(only_names_clf[i])][2]),'Catboost': string_to_float(apotelesmata_df_specificity[str(only_names_clf[i])][3]), 'AdaBoost': string_to_float(apotelesmata_df_specificity[str(only_names_clf[i])][4]), 'Bagging': string_to_float(apotelesmata_df_specificity[str(only_names_clf[i])][5]), 'ExtraTrees': string_to_float(apotelesmata_df_specificity[str(only_names_clf[i])][6]), 'RandomForest': string_to_float(apotelesmata_df_specificity[str(only_names_clf[i])][7]), 'Stacking': string_to_float(apotelesmata_df_specificity[str(only_names_clf[i])][8]), 'Voting': string_to_float(apotelesmata_df_specificity[str(only_names_clf[i])][9]), 'GradientBoosting': string_to_float(apotelesmata_df_specificity[str(only_names_clf[i])][10])}

 df_1 = pd.DataFrame(d_1)
 df_2 = pd.DataFrame(d_2)
 df_3 = pd.DataFrame(d_3)

 fig, ax = plt.subplots(1,3,figsize=(22.20,7.80)) # define the axis object here
 fig.suptitle(str(only_names_clf_new[i]))
 sns.set_style("whitegrid")

 sns.boxplot(ax=ax[0],data=df_1,showfliers=False,palette="Set1").set_title('accuracy')
 sns.boxplot(ax=ax[1],data=df_2,showfliers=False,palette="Set1").set_title('f1') 
 sns.boxplot(ax=ax[2],data=df_3,showfliers=False,palette="Set1").set_title('specificity') 


 plt.setp(ax[0].get_xticklabels(), rotation=25) #rotation of labels_x
 ax[0].yaxis.grid(False) # Hide the horizontal gridlines
 ax[0].xaxis.grid(True) # Show the vertical gridlines

 plt.setp(ax[1].get_xticklabels(), rotation=25) #rotation of labels_x
 ax[1].yaxis.grid(False) # Hide the horizontal gridlines
 ax[1].xaxis.grid(True) # Show the vertical gridlines
 
 plt.setp(ax[2].get_xticklabels(), rotation=25) #rotation of labels_x
 ax[2].yaxis.grid(False) # Hide the horizontal gridlines
 ax[2].xaxis.grid(True) # Show the vertical gridlines
    
 plt.savefig('C:/Users/John/Desktop/pros telos apo 14.06/boxplots/'+str(only_names_clf_new[i])+'.svg')

In [ ]:
#create comparison between algorithms
sigkrish={}
sigkrish['G3']=[0]*11
sigkrish['xgboost']=[0]*11
sigkrish['lightgbm']=[0]*11
sigkrish['catboost']=[0]*11
sigkrish['adaboost']=[0]*11
sigkrish['bagging']=[0]*11
sigkrish['extratrees']=[0]*11
sigkrish['randomforest']=[0]*11
sigkrish['stacking']=[0]*11
sigkrish['voting']=[0]*11
sigkrish['gradientboosting']=[0]*11



for i in range (26):
      d_1 = {'Voting_G3': string_to_float(apotelesmata_df_accuracy_score[str(only_names_clf[i])][0]),'XGBoost': string_to_float(apotelesmata_df_accuracy_score[str(only_names_clf[i])][1]), 'LightGBM': string_to_float(apotelesmata_df_accuracy_score[str(only_names_clf[i])][2]),'Catboost': string_to_float(apotelesmata_df_accuracy_score[str(only_names_clf[i])][3]), 'AdaBoost': string_to_float(apotelesmata_df_accuracy_score[str(only_names_clf[i])][4]), 'Bagging': string_to_float(apotelesmata_df_accuracy_score[str(only_names_clf[i])][5]), 'ExtraTrees': string_to_float(apotelesmata_df_accuracy_score[str(only_names_clf[i])][6]), 'RandomForest': string_to_float(apotelesmata_df_accuracy_score[str(only_names_clf[i])][7]), 'Stacking': string_to_float(apotelesmata_df_accuracy_score[str(only_names_clf[i])][8]), 'Voting': string_to_float(apotelesmata_df_accuracy_score[str(only_names_clf[i])][9]), 'GradientBoosting': string_to_float(apotelesmata_df_accuracy_score[str(only_names_clf[i])][10])}
      d_2= {'Voting_G3': string_to_float(apotelesmata_df_f1_score[str(only_names_clf[i])][0]),'XGBoost': string_to_float(apotelesmata_df_f1_score[str(only_names_clf[i])][1]), 'LightGBM': string_to_float(apotelesmata_df_f1_score[str(only_names_clf[i])][2]),'Catboost': string_to_float(apotelesmata_df_f1_score[str(only_names_clf[i])][3]), 'AdaBoost': string_to_float(apotelesmata_df_f1_score[str(only_names_clf[i])][4]), 'Bagging': string_to_float(apotelesmata_df_f1_score[str(only_names_clf[i])][5]), 'ExtraTrees': string_to_float(apotelesmata_df_f1_score[str(only_names_clf[i])][6]), 'RandomForest': string_to_float(apotelesmata_df_f1_score[str(only_names_clf[i])][6]), 'Stacking': string_to_float(apotelesmata_df_f1_score[str(only_names_clf[i])][8]), 'Voting': string_to_float(apotelesmata_df_f1_score[str(only_names_clf[i])][9]), 'GradientBoosting': string_to_float(apotelesmata_df_f1_score[str(only_names_clf[i])][10])}
      d_3={'Voting_G3': string_to_float(apotelesmata_df_specificity[str(only_names_clf[i])][0]),'XGBoost': string_to_float(apotelesmata_df_specificity[str(only_names_clf[i])][1]), 'LightGBM': string_to_float(apotelesmata_df_specificity[str(only_names_clf[i])][2]),'Catboost': string_to_float(apotelesmata_df_specificity[str(only_names_clf[i])][3]), 'AdaBoost': string_to_float(apotelesmata_df_specificity[str(only_names_clf[i])][4]), 'Bagging': string_to_float(apotelesmata_df_specificity[str(only_names_clf[i])][5]), 'ExtraTrees': string_to_float(apotelesmata_df_specificity[str(only_names_clf[i])][6]), 'RandomForest': string_to_float(apotelesmata_df_specificity[str(only_names_clf[i])][7]), 'Stacking': string_to_float(apotelesmata_df_specificity[str(only_names_clf[i])][8]), 'Voting': string_to_float(apotelesmata_df_specificity[str(only_names_clf[i])][9]), 'GradientBoosting': string_to_float(apotelesmata_df_specificity[str(only_names_clf[i])][10])}

      df_1 = pd.DataFrame(d_1)
      df_2 = pd.DataFrame(d_2)
      df_3 = pd.DataFrame(d_3)
      
      #pick df and set the max values of each column
      y=df_3.max()
      
      #to vhma gia na allazei h sigkrish
      p=0
      for x,t in sigkrish.items():
            
            if  y[p]>y[0] :
                sigkrish[x][0]=sigkrish[x][0]+1            
            if  y[p]>y[1] :
                sigkrish[x][1]=sigkrish[x][1]+1
            if y[p]>y[2]:
                sigkrish[x][2]=sigkrish[x][2]+1
            if y[p]>y[3]:
                sigkrish[x][3]=sigkrish[x][3]+1
            if y[p]>y[4]:
                sigkrish[x][4]=sigkrish[x][4]+1
            if y[p]>y[5]:
                sigkrish[x][5]=sigkrish[x][5]+1
            if y[p]>y[6]:
                sigkrish[x][6]=sigkrish[x][6]+1
            if y[p]>y[7]:
                sigkrish[x][7]=sigkrish[x][7]+1
            if y[p]>y[8]:
                sigkrish[x][8]=sigkrish[x][8]+1
            if y[p]>y[9]:
                sigkrish[x][9]=sigkrish[x][9]+1
            if y[p]>y[10]:
                sigkrish[x][10]=sigkrish[x][10]+1
                
            
            p=p+1

In [ ]:
# accuracy
sigkrish_1=sigkrish
sigkrish_1

In [ ]:
# f1_score
sigkrish_2=sigkrish
sigkrish_2

In [ ]:
# specificity
sigkrish_3=sigkrish
sigkrish_3

In [ ]:
df_sigkrish_1=pd.DataFrame.from_dict(sigkrish_1)
df_sigkrish_1 = df_sigkrish_1.T

df_sigkrish_2=pd.DataFrame.from_dict(sigkrish_2)
df_sigkrish_2 = df_sigkrish_2.T

df_sigkrish_3=pd.DataFrame.from_dict(sigkrish_3)
df_sigkrish_3 = df_sigkrish_3.T


df_sigkrish_1.to_excel('C:/Users/john/Desktop/machine learning/apotelesmata/df_sigkrish_1.xls')  
df_sigkrish_2.to_excel('C:/Users/john/Desktop/machine learning/apotelesmata/df_sigkrish_2.xls') 
df_sigkrish_3.to_excel('C:/Users/john/Desktop/machine learning/apotelesmata/df_sigkrish_3.xls') 